In [1]:
import copy
import os

# ignore bnb warnings
os.environ["BITSANDBYTES_NOWELCOME"] = "1"

if "RANK_TABLE_FILE" in os.environ:
    del os.environ["RANK_TABLE_FILE"]

In [2]:
import mindspore as ms
from mindspore.common.api import _no_grad
from mindnlp.core import nn, ops
import mindnlp.core.nn.functional as F
from mindnlp import peft

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

In [3]:
ms.manual_seed(0)
X = ops.rand((1000, 20))
y = (X.sum(1) > 10).long()

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [4]:
n_train = 800
batch_size = 64

In [5]:
X_train = ms.Tensor(X[:n_train])
y_train = ms.Tensor(y[:n_train])
train_dataset = ms.dataset.NumpySlicesDataset((X_train.asnumpy(), y_train.asnumpy()), column_names=["input_ids", "labels"], shuffle=True)
train_dataset = train_dataset.batch(batch_size)

X_eval = ms.Tensor(X[n_train:])
y_eval = ms.Tensor(y[n_train:])
eval_dataset = ms.dataset.NumpySlicesDataset((X_eval.asnumpy(), y_eval.asnumpy()), column_names=["input_ids", "labels"], shuffle=False)
eval_dataset = eval_dataset.batch(batch_size)

In [6]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[64, 20], dtype=Float32, value=
[[ 4.04855609e-01,  3.87673140e-01,  6.16844177e-01 ...  5.84283113e-01,  5.50715923e-01,  6.80523872e-01],
 [ 1.90978289e-01,  6.15762115e-01,  3.58265162e-01 ...  5.17028809e-01,  9.19216871e-02,  3.25543642e-01],
 [ 7.95297980e-01,  8.53119135e-01,  8.75666738e-01 ...  7.63129950e-01,  2.10702777e-01,  7.42496610e-01],
 ...
 [ 3.45470548e-01,  1.48276448e-01,  4.19310451e-01 ...  4.90973830e-01,  9.27417278e-01,  5.44965506e-01],
 [ 6.39575958e-01,  7.14566708e-01,  6.57585263e-01 ...  2.05765724e-01,  6.24854803e-01,  5.66110611e-02],
 [ 9.66328382e-01,  7.13570952e-01,  2.29314089e-01 ...  2.06972003e-01,  7.06779718e-01,  4.71898913e-01]]), 'labels': Tensor(shape=[64], dtype=Int64, value= [0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 
 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 
 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1])}


In [7]:
class MLP(nn.Module):
    def __init__(self, num_units_hidden=2000):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(20, num_units_hidden),
            nn.ReLU(),
            nn.Linear(num_units_hidden, num_units_hidden),
            nn.ReLU(),
            nn.Linear(num_units_hidden, 2),
            nn.LogSoftmax(dim=-1),
        )

    def forward(self, X):
        return self.seq(X)

In [8]:
lr = 0.002
batch_size = 64
num_epochs = 30

In [9]:
[(n, type(m)) for n, m in MLP().named_modules()]

[('', __main__.MLP),
 ('seq', mindnlp.core.nn.modules.container.Sequential),
 ('seq.0', mindnlp.core.nn.modules.linear.Linear),
 ('seq.1', mindnlp.core.nn.modules.activation.ReLU),
 ('seq.2', mindnlp.core.nn.modules.linear.Linear),
 ('seq.3', mindnlp.core.nn.modules.activation.ReLU),
 ('seq.4', mindnlp.core.nn.modules.linear.Linear),
 ('seq.5', mindnlp.core.nn.modules.activation.LogSoftmax)]

In [10]:
config = peft.LoraConfig(
    r=8,
    target_modules=["seq.0", "seq.2"],
    modules_to_save=["seq.4"],
)

In [11]:
from mindnlp.core import optim
from mindnlp.transformers.optimization import get_linear_schedule_with_warmup

module = MLP()
module_copy = copy.deepcopy(module)  # we keep a copy of the original model for later
for name, param in module.named_parameters():
    print(f"{name}: {param.shape}")

seq.0.weight: (2000, 20)
seq.0.bias: (2000,)
seq.2.weight: (2000, 2000)
seq.2.bias: (2000,)
seq.4.weight: (2, 2000)
seq.4.bias: (2,)


In [12]:
model = peft.get_peft_model(module, config)
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")

base_model.model.seq.0.base_layer.weight: (2000, 20)
base_model.model.seq.0.base_layer.bias: (2000,)
base_model.model.seq.0.lora_A.default.weight: (8, 20)
base_model.model.seq.0.lora_B.default.weight: (2000, 8)
base_model.model.seq.2.base_layer.weight: (2000, 2000)
base_model.model.seq.2.base_layer.bias: (2000,)
base_model.model.seq.2.lora_A.default.weight: (8, 2000)
base_model.model.seq.2.lora_B.default.weight: (2000, 8)
base_model.model.seq.4.original_cell.weight: (2, 2000)
base_model.model.seq.4.original_cell.bias: (2,)
base_model.model.seq.4.modules_to_save.default.weight: (2, 2000)
base_model.model.seq.4.modules_to_save.default.bias: (2,)


In [13]:
optimizer = optim.Adam([param for name, param in model.named_parameters() if "lora" in name], lr=lr)
criterion = nn.CrossEntropyLoss()
print(optimizer)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.002
    maximize: False
    weight_decay: 0
)


In [14]:
model.print_trainable_parameters()

trainable params: 52,162 || all params: 4,100,164 || trainable%: 1.2721930147184357


In [15]:
from tqdm import tqdm

def forward_fn(**batch):
    outputs = model(batch["input_ids"])
    loss = criterion(outputs, batch["labels"])
    return loss

grad_fn = ms.value_and_grad(forward_fn, None, model.trainable_params())

def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer.step(grads)
    return loss

for epoch in range(num_epochs):
    model.set_train(True)
    train_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        train_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with ms._no_grad():
            outputs = model(batch["input_ids"])
        loss = criterion(outputs, batch["labels"])
        eval_loss += loss.float()    

    eval_loss_total = (eval_loss / len(eval_dataset)).item()
    train_loss_total = (train_loss / len(train_dataset)).item()
    print(f"{epoch=:<2}  {train_loss_total=:.4f}  {eval_loss_total=:.4f}")

100%|██████████| 4/4 [00:00<00:00, 119.02it/s]


epoch=0   train_loss_total=0.6666  eval_loss_total=0.7073


100%|██████████| 4/4 [00:00<00:00, 212.61it/s]


epoch=1   train_loss_total=0.5501  eval_loss_total=0.6839


100%|██████████| 4/4 [00:00<00:00, 196.86it/s]


...

epoch=27  train_loss_total=0.0165  eval_loss_total=0.1244


100%|██████████| 4/4 [00:00<00:00, 218.47it/s]


epoch=28  train_loss_total=0.0166  eval_loss_total=0.1252


100%|██████████| 4/4 [00:00<00:00, 218.22it/s]

epoch=29  train_loss_total=0.0162  eval_loss_total=0.1259


In [16]:
for name, param in model.named_parameters():
    print(f"Parameter: {name} \t| Requires Grad: {param.requires_grad}")


Parameter: base_model.model.seq.0.base_layer.weight 	| Requires Grad: False
Parameter: base_model.model.seq.0.base_layer.bias 	| Requires Grad: False
Parameter: base_model.model.seq.0.lora_A.default.weight 	| Requires Grad: True
Parameter: base_model.model.seq.0.lora_B.default.weight 	| Requires Grad: True
Parameter: base_model.model.seq.2.base_layer.weight 	| Requires Grad: False
Parameter: base_model.model.seq.2.base_layer.bias 	| Requires Grad: False
Parameter: base_model.model.seq.2.lora_A.default.weight 	| Requires Grad: True
Parameter: base_model.model.seq.2.lora_B.default.weight 	| Requires Grad: True
Parameter: base_model.model.seq.4.original_cell.weight 	| Requires Grad: False
Parameter: base_model.model.seq.4.original_cell.bias 	| Requires Grad: False
Parameter: base_model.model.seq.4.modules_to_save.default.weight 	| Requires Grad: True
Parameter: base_model.model.seq.4.modules_to_save.default.bias 	| Requires Grad: True


In [17]:
for name, param in model.base_model.named_parameters():
    if "lora" not in name:
        continue

    print(f"New parameter {name:<13} | {param.numel():>5} parameters | updated")

New parameter model.seq.0.lora_A.default.weight |   160 parameters | updated
New parameter model.seq.0.lora_B.default.weight | 16000 parameters | updated
New parameter model.seq.2.lora_A.default.weight | 16000 parameters | updated
New parameter model.seq.2.lora_B.default.weight | 16000 parameters | updated
